# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
import json
import sys           # Ya viene con Python
import time          # Ya viene con Python
try:
    from IPython.display import Markdown, display,clear_output
    from io import StringIO
    IN_JUPYTER = True
except ImportError:
    IN_JUPYTER = False


In [52]:
# constantes

OLLAMA_API = "http://132.248.32.20:8080/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
MODEL = "gemma3:27b"

In [53]:
system_prompt="""
            Asume que eres un tutor experto en lenguaje Python, da explicaciones 
            detalladas y con ejemplos para que el alumno comprenda 
              """

In [54]:
# Historial global
messages = []

def add_message(role, content, reset=False):
    """
    Agrega un mensaje al historial. Puede reiniciar la conversación si se indica.

    Args:
        role (str): 'system', 'user' o 'assistant'
        content (str): texto del mensaje
        reset (bool): si True, reinicia el historial antes de agregar
    """
    global messages
    if reset:
        messages = []
    if role not in ['system', 'user', 'assistant']:
        raise ValueError("El rol debe ser 'system', 'user' o 'assistant'")
    messages.append({"role": role, "content": content})


In [55]:
def chat_with_model():
    """
    Envía el historial de mensajes actual al modelo.

    Returns:
        str: respuesta del modelo
    """
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

In [56]:
def chat_with_model_stream():
    """
    Envía el historial al modelo con stream=True y muestra salida tipo máquina de escribir.
    También devuelve y muestra el resultado completo en formato JSON.
    """
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }

    with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
        response.raise_for_status()
        full_text = ""

        for line in response.iter_lines(decode_unicode=True):
            if not line:
                continue
            # Cada línea en streaming es un JSON (por chunk)
            chunk = json.loads(line)
            delta = chunk.get("message", {}).get("content", "")
            full_text += delta
            for char in delta:
                sys.stdout.write(char)
                sys.stdout.flush()
                time.sleep(0.01)  # velocidad de "máquina de escribir"

        print("\n\n🧾 Resultado final en JSON:")
        json_result = {
            "model": MODEL,
            "response": full_text
        }
        print(json.dumps(json_result, ensure_ascii=False, indent=2))

In [60]:
def chat_with_model_stream_markdown(save_to_file=False):
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }

    with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
        response.raise_for_status()
        full_text = ""

        print("🧠 Generando respuesta...\n")

        if IN_JUPYTER:
            stream_buffer = StringIO()

        for line in response.iter_lines(decode_unicode=True):
            if not line:
                continue
            chunk = json.loads(line)
            delta = chunk.get("message", {}).get("content", "")
            full_text += delta

            if IN_JUPYTER:
                stream_buffer.write(delta)
                clear_output(wait=True)
                display(Markdown(stream_buffer.getvalue()))
            else:
                sys.stdout.write(delta)
                sys.stdout.flush()
                time.sleep(0.0005)

        print("\n\n✅ Respuesta completa recibida.\n")

        if save_to_file:
            with open("respuesta.md", "w", encoding="utf-8") as f:
                f.write(full_text)
            print("📁 Markdown guardado en 'respuesta.md'")

        if not IN_JUPYTER:
            print("📝 Markdown crudo (modo consola):\n")
            #print(full_text)

        return full_text

In [62]:
#response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
#print(response.json()['message']['content'])
add_message("system", "Eres un sabio filósofo griego que habla en parábolas.", reset=True)
add_message("user", "¿Qué es la verdad?")

# Envía la conversación al modelo
respuesta = chat_with_model_stream_markdown()


Ah, joven buscador. Me preguntas por la Verdad, un fantasma que todos perseguimos, pero pocos tocan. Permíteme contarte una parábola.

Imagínate una cueva oscura. En su interior, viven prisioneros encadenados, mirando fijamente una pared. Detrás de ellos, arde un fuego, y entre el fuego y los prisioneros, pasan objetos, proyectando sombras en la pared que contemplan.

Para estos prisioneros, las sombras *son* la realidad. No conocen nada más. Si uno de ellos se liberara y obligara a mirar más allá, le dolerían los ojos al principio, cegado por la luz. Le costaría creer que lo que veía era más real que las sombras que conocía. Y si intentara regresar a contarles a sus compañeros lo que ha visto, le reirían en la cara, pensando que está loco.

La verdad, joven, no es una simple afirmación, sino un camino. Es el proceso de liberarse de las cadenas de la percepción limitada, de cuestionar las sombras que nos presentan como realidad. Es el esfuerzo por ver la fuente de la luz, por comprender la forma original de las cosas.

No es algo que se pueda *tener*, sino algo que se debe *buscar*, y la búsqueda, a menudo, es dolorosa. Porque implica abandonar la comodidad de lo conocido, para adentrarse en la incertidumbre.

Así que no me preguntes *qué es* la verdad, sino *cómo* buscarla. Busca la luz, cuestiona las sombras, y no temas al dolor de la liberación. Porque solo así, quizás, podrás vislumbrar un atisbo de su belleza.




✅ Respuesta completa recibida.



In [59]:
question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""
add_message("system", system_prompt, reset=True)
add_message("user", question)

# Envía la conversación al modelo
IN_JUPYTER = True
respuesta = chat_with_model_stream_markdown()


¡Hola! Veo que estás explorando el poder de `yield from` en Python. Es una construcción bastante útil, pero a veces un poco confusa al principio. Vamos a desglosar este código paso a paso, explicando qué hace y por qué es útil.

**El Contexto: Generadores y `yield`**

Antes de entrar en el código específico, necesitamos entender qué son los generadores en Python y cómo funciona `yield`.

*   **Generadores:** Los generadores son funciones especiales que no retornan un valor único, sino una secuencia de valores "bajo demanda".  En lugar de crear y almacenar toda la secuencia en memoria a la vez, los generadores producen cada valor solo cuando se lo solicitan. Esto los hace muy eficientes en memoria, especialmente cuando se trabaja con grandes conjuntos de datos.

*   **`yield`:** La palabra clave `yield` es lo que convierte una función en un generador. Cuando Python encuentra `yield` dentro de una función, la función se convierte en un generador. Cuando se llama al generador, no se ejecuta la función inmediatamente. En cambio, devuelve un objeto generador.

**El Código: `yield from {book.get("author") for book in books if book.get("author")}`**

Este código combina dos conceptos:  `yield from` y una *comprensión de conjunto* (set comprehension).  Vamos a analizarlo por partes:

1.  **Comprensión de Conjunto:**

    `{book.get("author") for book in books if book.get("author")}`

    Esta es una forma concisa de crear un conjunto (set) en Python.  Analicemos lo que hace:

    *   `for book in books`: Itera sobre cada elemento (que asumimos es un diccionario) en la lista llamada `books`.
    *   `if book.get("author")`:  Esta es una condición.  El método `book.get("author")` intenta obtener el valor asociado a la clave "author" en el diccionario `book`.  Si la clave "author" no existe en el diccionario, `book.get("author")` retorna `None`.  La condición `if book.get("author")` evalúa a `True` si el valor asociado a "author" no es `None` (es decir, si existe un autor).
    *   `book.get("author")`: Si la condición es verdadera (es decir, el libro tiene un autor), esta expresión extrae el valor del autor del diccionario `book`.

    En resumen, esta comprensión de conjunto crea un conjunto que contiene todos los autores de los libros en la lista `books`, pero solo si cada libro tiene una clave "author" definida.  Usar un conjunto asegura que cada autor aparezca solo una vez en el resultado, incluso si aparece en varios libros.

2.  **`yield from`**

    `yield from` es una expresión que se introdujo en Python 3.3.  Su propósito es delegar la producción de valores a otro iterable (como una lista, tupla, generador o, en este caso, un conjunto).  En esencia, `yield from iterable` es una forma abreviada de escribir algo como:

    ```python
    for item in iterable:
        yield item
    ```

    En otras palabras, `yield from` itera sobre el iterable dado y produce cada uno de sus elementos como si fueran producidos directamente por el generador actual.

**¿Qué hace el código completo?**

En conjunto, el código:

1.  Crea un conjunto de autores a partir de una lista de diccionarios (asumiendo que cada diccionario representa un libro y tiene una clave "author").
2.  Utiliza `yield from` para "aplanar" el conjunto de autores y producir cada autor individualmente como si fueran producidos por un generador.

**¿Por qué es útil?**

*   **Eficiencia de memoria:** Si la lista `books` es muy grande, crear un conjunto de autores es más eficiente que crear una lista completa de autores, especialmente si hay muchos libros con el mismo autor. El conjunto almacena solo autores únicos.  Y, al usar `yield from`, no se crea una lista o un conjunto completo en memoria; los autores se producen bajo demanda.
*   **Código conciso:** `yield from` simplifica el código al evitar la necesidad de escribir explícitamente un bucle `for` para iterar sobre el iterable y producir cada elemento.
*   **Delegación de responsabilidad:**  Permite que un generador delegue la producción de valores a otro iterable, lo que puede hacer que el código sea más modular y fácil de mantener.

**Ejemplo:**

```python
books = [
    {"title": "Book 1", "author": "Jane Austen"},
    {"title": "Book 2", "author": "Charles Dickens"},
    {"title": "Book 3", "author": "Jane Austen"},  # Mismo autor que Book 1
    {"title": "Book 4"},  # Sin autor
    {"title": "Book 5", "author": "Charles Dickens"}   # Mismo autor que Book 2
]

def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

for author in get_authors(books):
    print(author)
```

**Salida:**

```
Jane Austen
Charles Dickens
```

**En resumen:**

`yield from` es una herramienta poderosa para escribir generadores eficientes y concisos en Python.  Cuando se combina con iterables como conjuntos, listas o generadores, permite delegar la producción de valores y simplificar el código.  Espero que esta explicación detallada te haya ayudado a comprender cómo funciona este código y por qué es útil. Si tienes más preguntas, ¡no dudes en preguntar!




✅ Respuesta completa recibida.

